In [19]:
from functools import lru_cache

import pandas as pd
import numpy as np

In [73]:
problem_input = """L.LL.LL.LL
LLLLLLL.LL
L.L.L..L..
LLLL.LL.LL
L.LL.LL.LL
L.LLLLL.LL
..L.L.....
LLLLLLLLLL
L.LLLLLL.L
L.LLLLL.LL"""

directions = (
    (-1, -1,),
    (-1, 0,),
    (-1, 1,),
    (0, -1,),
    (0, 1,),
    (1, -1,),
    (1, 0,),
    (1, 1,),
)

In [92]:
layout = pd.DataFrame(
    data=[list(line) for line in problem_input.splitlines()]
) \
    .replace({
        ".": np.nan,
        "L": 0,
        "#": 1
    })

layout

,0,1,2,3,4,5,6,7,8,9
0,0.0,NaN,0,0.0,NaN,0.0,0.0,NaN,0.0,0.0
1,0.0,0.0,0,0.0,0.0,0.0,0.0,NaN,0.0,0.0
2,0.0,NaN,0,NaN,0.0,NaN,NaN,0.0,NaN,NaN
3,0.0,0.0,0,0.0,NaN,0.0,0.0,NaN,0.0,0.0
4,0.0,NaN,0,0.0,NaN,0.0,0.0,NaN,0.0,0.0
5,0.0,NaN,0,0.0,0.0,0.0,0.0,NaN,0.0,0.0
6,NaN,NaN,0,NaN,0.0,NaN,NaN,NaN,NaN,NaN
7,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,NaN,0,0.0,0.0,0.0,0.0,0.0,NaN,0.0
9,0.0,NaN,0,0.0,0.0,0.0,0.0,NaN,0.0,0.0


In [64]:
@lru_cache(maxsize=None)
def find_next_seat(coord, direction):
    x, y = coord
    dx, dy = direction
    
    while True:
        x_, y_ = x+dx, y+dy
        
        if 0 <= x_ < layout.shape[0] and 0 <= y_ < layout.shape[1]:
            value = layout.iloc[x_, y_]
            if value in (0, 1):
                return value
            x, y = x_, y_

        else:
            break
    return np.nan

In [65]:
def get_surrounding_seats(coord):
    for d in directions:
        seat = find_next_seat(coord, d)
        if not np.isnan(seat):
            yield seat

In [66]:
def get_surrounding_seat_stats(surrounding_seats):
    surrounding_area_occupied = False
    surrounding_area_occupied_count = 0

    for seat in surrounding_seats:
        if seat == 1:
            surrounding_area_occupied = True
            surrounding_area_occupied_count += 1

    return surrounding_area_occupied, surrounding_area_occupied_count

In [67]:
def calc_new_value(
    value, 
    surrounding_area_occupied, 
    surrounding_area_occupied_count
):
    if value == 0 and not surrounding_area_occupied:
        return 1
    elif value == 1 and surrounding_area_occupied_count >= 5:
        return 0
    else:
        return value

In [68]:
def update_row(row):
    x = row.name
    new_row = pd.Series(np.zeros(layout.shape[1]))
    
    # index here is actually the column 
    for y in row.index:
        surrounding_seats = get_surrounding_seats((x, y))       
        
        new_value = calc_new_value(
            row[y],
            *get_surrounding_seat_stats(surrounding_seats)
        )
        
        new_row[y] = new_value
    return new_row

In [69]:
layout.apply(update_row, axis=1)

,0,1,2,3,4,5,6,7,8,9
0,1.0,NaN,1.0,1.0,NaN,1.0,1.0,NaN,1.0,1.0
1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,1.0,1.0
2,1.0,NaN,1.0,NaN,1.0,NaN,NaN,1.0,NaN,NaN
3,1.0,1.0,1.0,1.0,NaN,1.0,1.0,NaN,1.0,1.0
4,1.0,NaN,1.0,1.0,NaN,1.0,1.0,NaN,1.0,1.0
5,1.0,NaN,1.0,1.0,1.0,1.0,1.0,NaN,1.0,1.0
6,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN
7,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
8,1.0,NaN,1.0,1.0,1.0,1.0,1.0,1.0,NaN,1.0
9,1.0,NaN,1.0,1.0,1.0,1.0,1.0,NaN,1.0,1.0


In [93]:


for _ in range(100):
    find_next_seat.cache_clear()
    next_layout = layout.apply(update_row, axis=1)
    print("\n\n", next_layout)
    
    if layout.equals(next_layout):
        print(next_layout.astype(float).sum().sum())
        break
    else:
        layout = next_layout



      0    1    2    3    4    5    6    7    8    9
0  1.0  NaN  1.0  1.0  NaN  1.0  1.0  NaN  1.0  1.0
1  1.0  1.0  1.0  1.0  1.0  1.0  1.0  NaN  1.0  1.0
2  1.0  NaN  1.0  NaN  1.0  NaN  NaN  1.0  NaN  NaN
3  1.0  1.0  1.0  1.0  NaN  1.0  1.0  NaN  1.0  1.0
4  1.0  NaN  1.0  1.0  NaN  1.0  1.0  NaN  1.0  1.0
5  1.0  NaN  1.0  1.0  1.0  1.0  1.0  NaN  1.0  1.0
6  NaN  NaN  1.0  NaN  1.0  NaN  NaN  NaN  NaN  NaN
7  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0
8  1.0  NaN  1.0  1.0  1.0  1.0  1.0  1.0  NaN  1.0
9  1.0  NaN  1.0  1.0  1.0  1.0  1.0  NaN  1.0  1.0


      0    1    2    3    4    5    6    7    8    9
0  1.0  NaN  0.0  0.0  NaN  0.0  0.0  NaN  0.0  1.0
1  1.0  0.0  0.0  0.0  0.0  0.0  0.0  NaN  0.0  0.0
2  0.0  NaN  0.0  NaN  0.0  NaN  NaN  0.0  NaN  NaN
3  0.0  0.0  0.0  0.0  NaN  0.0  0.0  NaN  0.0  0.0
4  0.0  NaN  0.0  0.0  NaN  0.0  0.0  NaN  0.0  0.0
5  0.0  NaN  0.0  0.0  0.0  0.0  0.0  NaN  0.0  0.0
6  NaN  NaN  0.0  NaN  0.0  NaN  NaN  NaN  NaN  NaN
7  0.0

In [88]:
pd.DataFrame(data=[list(line) for line in """#.L#.L#.L#
#LLLLLL.LL
L.L.L..#..
##L#.#L.L#
L.L#.LL.L#
#.LLLL#.LL
..#.L.....
LLL###LLL#
#.LLLLL#.L
#.L#LL#.L#""".splitlines()]) \
    .replace({
        ".": np.nan,
        "L": 0,
        "#": 1
    }).astype(float).sum().sum()

26.0

In [91]:
next_layout.astype(float).sum().sum()

26.0